# GIS in Python - some examples
#### Computational Methods for Geoscience - Fall 2023
#### Instructor: Eric Lindsey
____


Will need GIS files: QFaults_GIS.zip
<br> https://www.usgs.gov/programs/earthquake-hazards/faults

In [ ]:
#pip install geopy

In [5]:
from geopy.geocoders import Nominatim
import panel as pn
import hvplot.pandas
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
#import pyproj
#import cartopy
#import geoviews

# the buffer option in geopandas has some annoying numpy warnings I want to hide
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

### Setting up New Environment

In [ ]:
# Lots of packages to install this time. 
# I think the easiest way is to use your terminal to run the following commands:

#conda create -n qgis python=3.10 anaconda -y # The last part selects 'yes'
#conda activate qgis
#conda install -c conda-forge jupyterlab panel geopy geopandas geoviews shapely cartopy hvplot

# this may take a little while...

### Use geopy's Nominatim package

In [6]:
# using geopy's Nominatim package to identify OpenStreetMap locations from free text input!
# see more in the docs - https://geopy.readthedocs.io/en/stable/
# this demo is based on some code from here: https://plugins.qgis.org/planet/tag/jupyter/

# create a connection to the Open Street Map API, called "Nominatim"
geolocator = Nominatim(user_agent='test_agent')

# enter any text string that you might search for in google maps:
address = "Northrop Hall"
location = geolocator.geocode(address)

# it found the location and address!
print(location.latitude,location.longitude)
print(location.address)

# try your own address!

35.082975000000005 -106.62286616271192
Northrop Hall, 221, Yale Boulevard Northeast, Nob Hill, Albuquerque, Bernalillo County, New Mexico, 87131, United States


With my address ...

In [7]:
# create a connection to the Open Street Map API, called "Nominatim"
geolocator = Nominatim(user_agent='test_agent')

# enter any text string that you might search for in google maps:
address = "1811 Girard Blvd NE, Albuquerque, NM"
location = geolocator.geocode(address)

# it found the location and address!
print(location.latitude,location.longitude)
print(location.address)

35.103787714285716 -106.61398151020408
1811, Girard Boulevard Northeast, Netherwood Park, Albuquerque, Bernalillo County, New Mexico, 87106, United States


### Create 'geopandas' dataframe from a lat, lon point

In [ ]:
# Check CRS
print(geocoded_gdf.crs)

In [11]:
# Create a "geopandas" dataframe from a lat,lon point. 
# Note, we must specify the coordinate system: epsg 4326 is a common one, which
# corresponds to the WGS84 ellipsoid with horizontal units of degrees.

geocoded_gdf = gpd.GeoDataFrame(pd.DataFrame([{
    'geometry': Point(location.longitude, location.latitude), 
    'address': address}])).set_crs('epsg:4326')

display(geocoded_gdf)

def hvplot_with_buffer(gdf, buffer, *args, **kwargs):
    # to use a buffer in meters we must first convert to a coordinate system that uses meters. 
    # epsg 3857 is a convenient one that is not great for data analysis but good for plotting - referred to as "web mercator" 
    # for the choice of map tiles ("CartoLight"), see more options here: https://holoviews.org/reference/elements/bokeh/Tiles.html
    buffered_gdf = gdf.to_crs('epsg:3857').buffer(buffer)
    buffered_gdf = gdf.copy().set_geometry(buffered_gdf).to_crs('epsg:4326')
    plot = ( buffered_gdf.hvplot(geo=True,  tiles='CartoLight', alpha=0.5, line_width=1 ) * 
             gdf.hvplot(geo=True, hover_cols=['DESIGNATION']) ).opts(active_tools=['wheel_zoom'])
    return plot

hvplot_with_buffer(geocoded_gdf, 1000)

,geometry,address
0,POINT (-106.61398 35.10379),"1811 Girard Blvd NE, Albuquerque, NM"


Exception: An axis may only be assigned one projection type

:Overlay
   .Tiles.I    :Tiles   [x,y]
   .Polygons.I :Polygons   [Longitude,Latitude]
   .Points.I   :Points   [Longitude,Latitude]

In [10]:
# let's use "Panel" to create a neat web-app-like demo:
import panel as pn

geolocator = Nominatim(user_agent='test_agent')

def my_plot(address="University of New Mexico", buffer_meters=1000):
    location = geolocator.geocode(address)
    geocoded_gdf = gpd.GeoDataFrame(pd.DataFrame([
        {'geometry': Point(location.longitude, location.latitude), 'address': address}
        ])).set_crs('epsg:4326')
    map_plot = hvplot_with_buffer(geocoded_gdf, buffer_meters, title=f'Geocoded address with {buffer_meters}m buffer')
    return map_plot.opts(active_tools=['wheel_zoom']) 

# specify the default arguments:
kw = dict(address="University of New Mexico", buffer_meters=(1,10000))

# similar to the "interactive" library we've been using, but a bit more complex
pn.interact(my_plot, **kw)
pn.template.FastListTemplate(
    site="Panel", title="Geocoding Demo", 
    main=[pn.interact(my_plot, **kw)]
).servable();

# now, click the little blue slider-bars icon at the top of your notebook.
# It should open up the same map as above, but in its own tab within your jupyter notebook. Super neat!
# check out tons of cool demos on the panel page - https://panel.holoviz.org/gallery/index.html

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=University+of+New+Mexico&format=json&limit=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7faab7dbbfd0>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=1)'))

### ShapeFile with geopandas

In [ ]:
# loading shapefiles is easy with geopandas
filename = '/Users/Adrian/OneDrive - University of New Mexico/EPS 522/Labs/Lab 11 GIS/Qfaults_GIS/SHP/Qfaults_US_Database.shp'
qfaults_gdf = gpd.read_file(filename)
display(qfaults_gdf)

# note, this is a large file and takes a while to load...

### geopandas tutorial

geopandas has some neat mapping capabilities for working with geographic data. Check out the tutorial:
<br> https://geopandas.org/en/stable/docs/user_guide/mapping.html

#### Plot of fault data over New Mexico

In [ ]:
# here is a quick plot of the data we loaded above.
# to reduce the number of faults to plot, we select only the ones in New Mexico.
nm_faults = qfaults_gdf[qfaults_gdf['Location'] == 'New Mexico']

nm_faults.plot()

# you could add these faults to any other figure you are working with already.

In [ ]:
# we can do even better with the .explore() option - check it out!
nm_faults.explore()


#### GIS computations of our data

In [ ]:
# Finally, let's do some GIS computations on our data. 
# We have the latitude and longitude of our address from above, what's the nearest fault to this location?

nearest = gpd.sjoin_nearest(geocoded_gdf,nm_faults,distance_col="distance to fault (deg)")
display(nearest)

# if you are using Northrop Hall, the nearest fault is probably "McCormick Ranch Faults", which is just south of the airport. Check it out in the map above.

In [ ]:
# We got a warning that we are doing distance calculations in a geographic CRS, which could lead to errors. Let's fix this:
# I want to use an equidistant conic projection, which has the ESRI number 102005.
# However, python does not know this number, so we need to define the projection.
# from the page here: https://spatialreference.org/ref/esri/usa-contiguous-equidistant-conic/
# I found the "Proj.4" definition as follows:
# +proj=eqdc +lat_0=0 +lon_0=0 +lat_1=33 +lat_2=45 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs 

my_crs = pyproj.CRS('+proj=eqdc +lat_0=0 +lon_0=0 +lat_1=33 +lat_2=45 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs')

nearest = gpd.sjoin_nearest(geocoded_gdf.to_crs(my_crs),nm_faults.to_crs(my_crs),distance_col="distance to fault (m)")
display(nearest)


In [ ]:
# now the location of our address is in the new projected CRS, which isn't that meaningful. Luckily we can transform back:
display(nearest.to_crs('epsg:4326'))

## Optional assignment:

Use what you've learned above (and your own google-fu) to accomplish the following task:

Create an interactive Panel that allows a user to enter their address, then shows the address on the map along with the nearest fault, and displays a message stating the distance to the fault.